In [18]:
# pip install lightgbm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 80.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import random
import os
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, StackingRegressor

from sklearn.model_selection import GridSearchCV
from math import pi
from math import log
from math import e
import math
import itertools
import csv
from lightgbm import LGBMRegressor
import matplotlib.pyplot as plt
import collections

In [2]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(41)

In [3]:
column_list = ['start_end_prob_s', 'start_end_prob_s1', 'start_end_prob_y', '면적', '부피', '수목', 'start_end_prob_s2', 'start_end_prob_s3', 'start_end_prob_s4', 'start_end_prob_s5\
', 'start_end_prob_s6', 'start_end_prob_s7', 'start_end_prob_s8', 'start_end_prob_s9', 'start_end_prob_s10', 'start_end_prob_s11', 'start_end_prob_s12']
# column_list는 넣고싶은 모든 컬럼명입니다
print(len(a))

17


In [4]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')
train_org = train.reset_index().copy()
test_org = test.reset_index().copy()
display(train.head())

,ID,착과량(int),수고(m),수관폭1(min),수관폭2(max),수관폭평균,2022-09-01 새순,2022-09-02 새순,2022-09-03 새순,2022-09-04 새순,...,2022-11-19 엽록소,2022-11-20 엽록소,2022-11-21 엽록소,2022-11-22 엽록소,2022-11-23 엽록소,2022-11-24 엽록소,2022-11-25 엽록소,2022-11-26 엽록소,2022-11-27 엽록소,2022-11-28 엽록소
0,TRAIN_0000,692,275.0,287.0,292.0,289.5,2.8,2.8,2.7,2.7,...,70.978249,70.876794,70.705253,70.559603,70.427356,70.340491,70.293830,70.262422,70.169841,70.043251
1,TRAIN_0001,534,293.0,284.0,336.0,310.0,3.3,3.3,3.3,3.2,...,71.535483,71.382303,71.253604,71.092665,70.955608,70.796630,70.597550,70.565088,70.560502,70.427600
2,TRAIN_0002,634,300.0,392.0,450.0,421.0,3.0,2.9,2.9,2.9,...,71.279804,71.199570,71.144020,71.026740,70.920038,70.876723,70.710129,70.595971,70.418203,70.399578
3,TRAIN_0003,639,289.0,368.0,379.0,373.5,3.1,3.0,3.0,3.0,...,69.934615,69.884124,69.845683,69.794682,69.779813,69.614644,69.455404,69.281590,69.238689,69.133970
4,TRAIN_0004,496,306.0,353.0,358.0,355.5,3.7,3.6,3.6,3.6,...,68.313016,68.285364,68.209860,68.209458,68.040083,67.859963,67.775556,67.701456,67.504244,67.410093


In [6]:
# train['원기둥부피'] = (((train['수관폭평균'] / 2) ** 2) * pi) * train['수고(m)']
# test['원기둥부피'] = (((test['수관폭평균'] / 2) ** 2) * pi) * test['수고(m)']

In [7]:
# train['구부피'] = (((train['수관폭평균'] / 2) ** 3) * pi) * 4/3
# test['구부피'] = (((test['수관폭평균'] / 2) ** 3) * pi) * 4/3

In [8]:
# train['햇빛받는잎사귀넓이'] = (((train['수관폭평균'] / 2) ** 2) * pi) * 4 / 2
# test['햇빛받는잎사귀넓이'] = (((test['수관폭평균'] / 2) ** 2) * pi) * 4 / 2

In [9]:
train['면적'] = train['수관폭평균'] ** 2
test['면적'] = test['수관폭평균'] ** 2

In [10]:
# train['test'] = train['수관폭평균'] / train['start_end_prob_y']

In [11]:
train['start_end_prob_s'] = train['2022-11-28 새순'] / train['2022-09-01 새순']
test['start_end_prob_s'] = test['2022-11-28 새순'] / test['2022-09-01 새순']


train['start_end_prob_s1'] = train['2022-11-20 새순'] / train['2022-11-01 새순'] * train['2022-09-01 새순'] * train['2022-10-01 새순'] ** 1.7
test['start_end_prob_s1'] = test['2022-11-01 새순'] / test['2022-09-01 새순']

# train['test11'] = train['start_end_prob_s1'] ** 2

train['start_end_prob_y'] = train['2022-11-28 엽록소'] / train['2022-09-01 엽록소']
test['start_end_prob_y'] = test['2022-11-28 엽록소'] / test['2022-09-01 엽록소']

train['부피'] = train['수관폭1(min)'] ** 2 * train['수고(m)']
test['부피'] = test['수관폭1(min)'] ** 2 * test['수고(m)']

train['start_end_prob_s1'] = train['2022-09-07 새순'] / train['2022-09-01 새순']
test['start_end_prob_s1'] = test['2022-09-07 새순'] / test['2022-09-01 새순']

train['start_end_prob_s2'] = train['2022-09-14 새순'] / train['2022-09-08 새순']
test['start_end_prob_s2'] = test['2022-09-14 새순'] / test['2022-09-08 새순']

train['start_end_prob_s3'] = train['2022-09-21 새순'] / train['2022-09-15 새순']
test['start_end_prob_s3'] = test['2022-09-21 새순'] / test['2022-09-15 새순']

train['start_end_prob_s4'] = train['2022-09-28 새순'] / train['2022-09-22 새순']
test['start_end_prob_s4'] = test['2022-09-28 새순'] / test['2022-09-22 새순']

train['start_end_prob_s5'] = train['2022-10-05 새순'] / train['2022-09-29 새순']
test['start_end_prob_s5'] = test['2022-10-05 새순'] / test['2022-09-29 새순']

train['start_end_prob_s6'] = train['2022-10-12 새순'] / train['2022-10-06 새순']
test['start_end_prob_s6'] = test['2022-10-12 새순'] / test['2022-10-06 새순']

train['start_end_prob_s7'] = train['2022-10-19 새순'] / train['2022-10-13 새순']
test['start_end_prob_s7'] = test['2022-10-19 새순'] / test['2022-10-13 새순']

train['start_end_prob_s8'] = train['2022-10-26 새순'] / train['2022-10-20 새순']
test['start_end_prob_s8'] = test['2022-10-26 새순'] / test['2022-10-20 새순']

train['start_end_prob_s9'] = train['2022-11-02 새순'] / train['2022-10-27 새순']
test['start_end_prob_s9'] = test['2022-11-02 새순'] / test['2022-10-27 새순']

train['start_end_prob_s10'] = train['2022-11-09 새순'] / train['2022-11-03 새순']
test['start_end_prob_s10'] = test['2022-11-09 새순'] / test['2022-11-03 새순']

train['start_end_prob_s11'] = train['2022-11-16 새순'] / train['2022-11-10 새순']
test['start_end_prob_s11'] = test['2022-11-16 새순'] / test['2022-11-10 새순']

train['start_end_prob_s12'] = train['2022-11-23 새순'] / train['2022-11-17 새순']
test['start_end_prob_s12'] = test['2022-11-23 새순'] / test['2022-11-17 새순']

# # train['start_end_prob_s13'] = train['2022-11-28 새순'] / train['2022-11-24 새순']
# # test['start_end_prob_s13'] = test['2022-11-28 새순'] / test['2022-11-24 새순']

# train['start_end_prob_y1'] = train['2022-09-07 엽록소'] / train['2022-09-01 엽록소']
# test['start_end_prob_y1'] = test['2022-09-07 엽록소'] / test['2022-09-01 엽록소']

# train['start_end_prob_y2'] = train['2022-09-14 엽록소'] / train['2022-09-08 엽록소']
# test['start_end_prob_y2'] = test['2022-09-14 엽록소'] / test['2022-09-08 엽록소']

# train['start_end_prob_y3'] = train['2022-09-21 엽록소'] / train['2022-09-15 엽록소']
# test['start_end_prob_y3'] = test['2022-09-21 엽록소'] / test['2022-09-15 엽록소']

# train['start_end_prob_y4'] = train['2022-09-28 엽록소'] / train['2022-09-22 엽록소']
# test['start_end_prob_y4'] = test['2022-09-28 엽록소'] / test['2022-09-22 엽록소']

# train['start_end_prob_y5'] = train['2022-10-05 엽록소'] / train['2022-09-29 엽록소']
# test['start_end_prob_y5'] = test['2022-10-05 엽록소'] / test['2022-09-29 엽록소']

# train['start_end_prob_y6'] = train['2022-10-12 엽록소'] / train['2022-10-06 엽록소']
# test['start_end_prob_y6'] = test['2022-10-12 엽록소'] / test['2022-10-06 엽록소']

# train['start_end_prob_y7'] = train['2022-10-19 엽록소'] / train['2022-10-13 엽록소']
# test['start_end_prob_y7'] = test['2022-10-19 엽록소'] / test['2022-10-13 엽록소']

# train['start_end_prob_y8'] = train['2022-10-26 엽록소'] / train['2022-10-20 엽록소']
# test['start_end_prob_y8'] = test['2022-10-26 엽록소'] / test['2022-10-20 엽록소']

# train['start_end_prob_y9'] = train['2022-11-02 엽록소'] / train['2022-10-27 엽록소']
# test['start_end_prob_y9'] = test['2022-11-02 엽록소'] / test['2022-10-27 엽록소']

# train['start_end_prob_y10'] = train['2022-11-09 엽록소'] / train['2022-11-03 엽록소']
# test['start_end_prob_y10'] = test['2022-11-09 엽록소'] / test['2022-11-03 엽록소']

# train['start_end_prob_y11'] = train['2022-11-16 엽록소'] / train['2022-11-10 엽록소']
# test['start_end_prob_y11'] = test['2022-11-16 엽록소'] / test['2022-11-10 엽록소']

# train['start_end_prob_y12'] = train['2022-11-23 엽록소'] / train['2022-11-17 엽록소']
# test['start_end_prob_y12'] = test['2022-11-23 엽록소'] / test['2022-11-17 엽록소']

# train['start_end_prob_y13'] = train['2022-11-28 엽록소'] / train['2022-11-24 엽록소']
# test['start_end_prob_y13'] = test['2022-11-28 엽록소'] / test['2022-11-24 엽록소']

In [12]:
train['수목'] = train['수고(m)'] * train['수관폭평균']
test['수목'] = test['수고(m)'] * test['수관폭평균']

In [216]:
new_soon = [s for s in train if "새순" in s]
greengreen = [s for s in train if "엽록" in s]
new_soon1 = [s for s in test if "새순" in s]
greengreen1 = [s for s in test if "엽록" in s]

tr_green = train[greengreen]
tr_saesoon = train[new_soon]
ts_green = test[greengreen1]
ts_saesoon = test[new_soon1]

train['엽록소기울기'] = (tr_green.iloc[:, 70] - tr_green.iloc[:, 0]) / 70
train['새순기울기'] = (tr_saesoon.iloc[:, 70] - tr_saesoon.iloc[:, 0]) / 70
test['엽록소기울기'] = (ts_green.iloc[:, 70] - ts_green.iloc[:, 0]) / 70
test['새순기울기'] = (ts_saesoon.iloc[:, 70] - ts_saesoon.iloc[:, 0]) / 70

In [ ]:
def NMAE(true, pred):
    mae = np.mean(np.abs(true-pred))
    score = mae / np.mean(np.abs(true))
    return score

In [ ]:
column_list = ['start_end_prob_s', 'start_end_prob_s1', 'start_end_prob_y', '면적', '부피', '수목', 'start_end_prob_s2', 'start_end_prob_s3', 'start_end_prob_s4', 'start_end_prob_s5\
', 'start_end_prob_s6', 'start_end_prob_s7', 'start_end_prob_s8', 'start_end_prob_s9', 'start_end_prob_s10', 'start_end_prob_s11', 'start_end_prob_s12']

model_list = {
    'rf': RandomForestRegressor(n_estimators = 195),
    'gb': GradientBoostingRegressor(learning_rate = 0.03, n_estimators = 145),
    'lgbm': LGBMRegressor()
}
ts_size = 0.2
rad_state = 42
shuf = True
drop_col = ['ID']
target = ['착과량(int)']

In [244]:
NMAEE = []
feature = []
pred_list = []

def similar_autoML(mint, maax):
    '''
    mint = 최소 컬럼 수, maax = 최대 컬럼수
    
    '''
    for i in range (mint, maax + 1):
        caselist = list(itertools.combinations(col_list, i))
        list(caselist)
        for n in range (len(caselist)):
            feature.append(list(caselist[n]))
            train_test = train[list(caselist[n])].reset_index(drop = True)
            result1 = pd.concat([train_org,train_test], axis = 1)
            result2 = result1.drop([drop_col], axis=1)
            x = result2.drop(target, axis=1)
            y = result2[target]
            x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=ts_size, random_state=rad_state, shuffle=shuf)
            # print(x_train.shape)
            # print(collections.Counter(list(train.isna().sum())))
            
            ###################################################################################################################
            
            models = model_list

            stacking = StackingRegressor(
                estimators=list(models.items()),
                final_estimator=LinearRegression(),
                cv=5
            )
            print(str(list(caselist[n])), '진행중')
            stacking.fit(x_train, y_train)
            
            ###################################################################################################################
            
            pred_stacking = stacking.predict(x_valid)
            
            pred_list.append(pred_stacking)
            
            nmae_mandarin = NMAE(y_valid, pred_stacking)
            
            print('NMAE:', nmae_mandarin)
            
            NMAEE.append(nmae_mandarin)
            
            with open('NMAE.csv','w',newline='') as f:
                writer = csv.writer(f)
                writer.writerow(NMAEE)
                
            with open('feature_name.csv','w',newline='') as f:
                writer = csv.writer(f)
                writer.writerow(feature)
                
            with open('pred.csv','w',newline='') as f:
                writer = csv.writer(f)
                writer.writerow(pred_stacking)
                
    return nmae_mandarin, NMAEE, pred_list

(1655, 200)
Counter({0: 201})
['start_end_prob_s', 'start_end_prob_s1', 'start_end_prob_y', '면적', '부피', '수목', 'start_end_prob_s2', 'start_end_prob_s3', 'start_end_prob_s4', 'start_end_prob_s5', 'start_end_prob_s6', 'start_end_prob_s7', 'start_end_prob_s8', 'start_end_prob_s9', 'start_end_prob_s10', 'start_end_prob_s11', 'start_end_prob_s12']진행중
NMAE: 0.07711301742485632


In [ ]:
similar_autoML(1, len(column_list))